<a href="https://colab.research.google.com/github/ChicagoPark/Basic_LiDAR_Process/blob/master/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Necessary Modules
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import os
from keras.preprocessing.image import ImageDataGenerator # Data Augmentation(Image transformations) 을 하기 위해 필요한 모듈

import scipy
import keras
from tensorflow.keras.utils import plot_model
! pip install keras_tuner
import keras_tuner as kt
import shutil
! pip install tensorflow-gpu

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [ ]:
# Colab 상의 훈련용 데이터 디렉터리 정리
!mkdir training_data
%cd training_data/
!mkdir '0. stone'
!mkdir '1. pothole'
!mkdir '2. filled'
%cd ..

/content/training_data
/content


In [ ]:
'''
# data path
# 인터넷에서 주워온것임 ! gdown --id 1PGhXUnaJDpcjgoLEhNn9gRvGZGRCzWUt
'''

# 빅데이터 AI 수업에서 공유된 data.zip 
! gdown --id 1X1J2RwBu9KhRpw8GqS-s1ZeI6YF0VZTO --out 'data.zip'

# 빅데이터 AI 수업에서 공유된 실차 데이터.zip
! gdown --id 1byD-TlXFXVJysi7q3K8y-W_96qbbfF5L --out '실차데이터.zip'

# json parsing 에서 나누었던 데이터.zip
! gdown --id 1Dr8z1zXNpTZVqcKd1cYn0d71gi5itA3W --out 'divided_data.zip'

# 직접 데이터 추출 및 작업을 진행한 구글드라이브 상 데이터
# ! gdown --id <추가예정> --out '<설정하고자하는폴더명>.zip'

'''
data.zip 파일은 큰 파일이니 아래 절차로 zip 해제할 것
! zip -FFv data.zip --out data_.zip
! unzip data_.zip -d data
'''

!unzip '실차데이터.zip' -d '실차데이터'
!unzip 'divided_data.zip' -d 'divided_data'

! zip -FFv data.zip --out data_.zip
! unzip data_.zip -d data

Permission denied: https://drive.google.com/uc?id=1X1J2RwBu9KhRpw8GqS-s1ZeI6YF0VZTO
Maybe you need to change permission over 'Anyone with the link'?
Downloading...
From: https://drive.google.com/uc?id=1byD-TlXFXVJysi7q3K8y-W_96qbbfF5L
To: /content/실차데이터.zip
100% 462M/462M [00:01<00:00, 241MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Dr8z1zXNpTZVqcKd1cYn0d71gi5itA3W
To: /content/divided_data.zip
100% 890M/890M [00:06<00:00, 131MB/s]
Archive:  실차데이터.zip
  inflating: 실차데이터/0. no_pothole/100490.png  
  inflating: 실차데이터/0. no_pothole/100880.png  
  inflating: 실차데이터/0. no_pothole/101490.png  
  inflating: 실차데이터/0. no_pothole/10170.png  
  inflating: 실차데이터/0. no_pothole/102100.png  
  inflating: 실차데이터/0. no_pothole/102450.png  
  inflating: 실차데이터/0. no_pothole/102810.png  
  inflating: 실차데이터/0. no_pothole/103340.png  
  inflating: 실차데이터/0. no_pothole/103800.png  
  inflating: 실차데이터/0. no_pothole/104080.png  
  inflating: 실차데이터/0. no_pothole/104700.png  
  inflating: 실차데이터/0. no_

In [ ]:
! gdown --id 1FB_Ssfk9A06wUFK5TlKN5tjKHaGOPsAA --out 'data.zip'
! zip -FFv data.zip --out data_.zip
! unzip data_.zip -d data

Permission denied: https://drive.google.com/uc?id=1FB_Ssfk9A06wUFK5TlKN5tjKHaGOPsAA
Maybe you need to change permission over 'Anyone with the link'?
Fix archive (-FF) - salvage what can
	zip warning: could not open input archive: data.zip
Scanning for entries...


Could not find:
  data.z01

Hit c      (change path to where this split file is)
    s      (skip this split)
    q      (abort archive - quit)
    e      (end this archive - no more splits)
    z      (look for .zip split - the last split)
 or ENTER  (try reading this split again): q
	zip warning: could not find split: data.z01
	zip warning: zip file empty
Archive:  data_.zip
warning [data_.zip]:  zipfile is empty


In [ ]:
# training_data 폴더로 zip 해제한 모든 파일을 정리한다.
!mv /content/실차데이터/'0. no_pothole'/* /content/training_data/'0. stone'
!mv /content/실차데이터/'1. pothole'/* /content/training_data/'1. pothole'
!mv /content/실차데이터/'2. filled'/* /content/training_data/'2. filled'

!mv /content/divided_data/'0. stone/Images'/* /content/training_data/'0. stone'
!mv /content/divided_data/'1. pothole/Images'/* /content/training_data/'1. pothole'
!mv /content/divided_data/'2. filled/Images'/* /content/training_data/'2. filled'

!mv /content/data/data/'0. stone'/* /content/training_data/'0. stone'
!mv /content/data/data/'1. pothole'/* /content/training_data/'1. pothole'
!mv /content/data/data/'2. filled'/* /content/training_data/'2. filled'

mv: cannot stat '/content/data/data/0. stone/*': No such file or directory
mv: cannot stat '/content/data/data/1. pothole/*': No such file or directory
mv: cannot stat '/content/data/data/2. filled/*': No such file or directory


In [ ]:
# 1. 기본 뿌리 디렉터리와 검증데이터 경로를 가져온다
base_path = os.path.abspath('/content/training_data')
print(base_path)

# 2. 데이터 경로를 지정을 해준다.
no_pothole_dir = os.path.join(base_path, '0. stone')
pothole_dir = os.path.join(base_path, '1. pothole')
filled_dir = os.path.join(base_path, '2. filled')

# 3. 데이터 셋의 개수를 카운팅한다.
num_no_pothole = len(os.listdir(no_pothole_dir))
num_pothole_dir = len(os.listdir(pothole_dir))
num_filled_dir = len(os.listdir(filled_dir))

total_data = num_no_pothole + num_pothole_dir + num_filled_dir

print(f'정상도로 : {num_no_pothole} 개 , 팟홀도로 : {num_pothole_dir}, 보수도로 : {num_filled_dir}')

print(f'Total Data 개수 : {total_data}')

/content/training_data
정상도로 : 172 개 , 팟홀도로 : 322, 보수도로 : 523
Total Data 개수 : 1017


In [ ]:
# Train set 과 Validation set 을 나누며, Data Augmentation 작업 또한 추가시킨다.
#11/12 여기 배치 정규화 추가하는 코드 찾으면 넣자
train_image_generator = ImageDataGenerator(rescale=1./255,
      rotation_range=8,
      width_shift_range=0.1,
      height_shift_range=0.1,
      shear_range=0.1,
      zoom_range=0.1,
      horizontal_flip=True,
      fill_mode='nearest',
      validation_split=0.15)
validation_image_generator = ImageDataGenerator(rescale=1./255,
                                                validation_split=0.15)
batch_size = 32
epochs = 15
IMG_HEIGHT = 320
IMG_WIDTH = 320

In [ ]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=base_path,
                                                           seed = 42,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           subset='training',
                                                           class_mode='sparse')

val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=base_path,
                                                           seed = 42,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           subset='validation',
                                                           class_mode='sparse')
train_num = train_data_gen.samples # training set 이 몇개인지
val_num = val_data_gen.samples   # validation set 이 몇개인지

Found 866 images belonging to 3 classes.
Found 151 images belonging to 3 classes.


# Model Loading

In [ ]:
# Create base model
# input_tensor 는 내가 쓰고 싶은 입력크기
# input_shape 는 모델이 가지는 입력크기를 말하는듯

def build_model(hp):
    base_model = tf.keras.applications.Xception(
        include_top=False,
        weights="imagenet",
        input_tensor=None,
        input_shape=None)

    # 해당 층의 매개변수는 그대로 사용하기 위해서 freezing 을 한다.
    base_model.trainable = False
    #base_model.trainable = False  # 기존에 false 였음
    inputs = keras.Input(shape=(320, 320, 3))
    x = base_model(inputs, training=False)
    #x = keras.layers.Dropout(0.2)(x)
    x=keras.layers.GlobalAveragePooling2D()(x)
    
    # Tune the number of layers.
    for i in range(hp.Int("num_layers", 1, 5)):
        x=keras.layers.Dense(# Tune number of units separately.
                units=hp.Int(f"units_{i}", min_value=32, max_value=512, step=32),
                activation=hp.Choice("activation", ["relu", "elu"]))(x)
    if hp.Boolean("dropout"):
        x = keras.layers.Dropout(0.25)(x)
    outputs = keras.layers.Dense(3, activation='softmax')(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    # Define the optimizer learning rate as a hyperparameter.
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(), # 교차엔트로피로 구성
              metrics=['accuracy'])
    
    return model


In [ ]:
# Keras_tuner 가 정상 작동하는지 확인
build_model(kt.HyperParameters())

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AddV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task

In [ ]:
tuner = kt.BayesianOptimization(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=4,
    executions_per_trial=2,
    overwrite=True,
    directory="my_dir",
    project_name="helloworld",
)

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AddV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task

In [ ]:
tuner.search_space_summary()

Search space summary
Default search space size: 5
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 5, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'elu'], 'ordered': False}
dropout (Boolean)
{'default': False, 'conditions': []}
lr (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

tuner.search(train_data_gen,validation_data=val_data_gen, epochs=3)


Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
num_layers        |5                 |?                 
units_0           |32                |?                 
activation        |relu              |?                 
dropout           |True              |?                 
lr                |0.0001928         |?                 

Executing op DestroyResourceOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op DestroyResourceOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op DestroyResourceOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op DestroyResourceOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op DestroyResourceOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op DestroyResourceOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op DestroyResourceOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op DestroyResource

KeyboardInterrupt: ignored

In [ ]:
tuner.results_summary()

Results summary
Results in my_dir/helloworld
Showing 10 best trials
Objective(name='val_accuracy', direction='max')


In [ ]:
# Create base model
# input_tensor 는 내가 쓰고 싶은 입력크기
# input_shape 는 모델이 가지는 입력크기를 말하는듯



strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
  base_model = tf.keras.applications.MobileNetV2(
    input_shape=None,
    alpha=1.0,
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    pooling=None
  )

# 해당 층의 매개변수는 그대로 사용하기 위해서 freezing 을 한다.
  base_model.trainable = True

  inputs = keras.Input(shape=(320, 320, 3))
  x = base_model(inputs, training=True)
  #x = keras.layers.Dropout(0.2)(x)
  x=keras.layers.GlobalAveragePooling2D()(x)
  x=keras.layers.Dense(64,activation='elu')(x)
  x = keras.layers.Dropout(0.4)(x)
  x=keras.layers.Dense(32,activation='elu')(x)
  x = keras.layers.Dropout(0.4)(x)
  outputs = keras.layers.Dense(3, activation='softmax')(x)

  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AddV2 in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op NoOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:

In [ ]:
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=2),
    tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
]

In [ ]:
tf.debugging.set_log_device_placement(False)

In [ ]:
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=train_num//batch_size,     # 한 프로퍼게이션때 한 배치가 쭉 통과하므로 그렇군
    epochs= 3,
    validation_data = val_data_gen,
    validation_steps = val_num//batch_size,
    callbacks=[my_callbacks]
)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  import sys


Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op TensorDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op DatasetCardinality in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ComputeBatchSize in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op NotEqual in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RebatchDatase

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Ide